# Vanishing Sphere (Reducing radius over time)
This Notebook launches a condition number study for the vanishing sphere testcase as presented in the paper "Cell agglomeration strategy for eXtended discontinous Galerkin methods" (2024) 
 The calculations can be very long, in order to use a HPC cluster ``..\.BoSSS\etc\BatchProcessorConfig.json`` must be modified for the execution queue.

Initiliaze BoSSS

In [1]:
#r "./../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
buildname = String.IsNullOrEmpty(buildname)? "VanishingSphere2D" : buildname;
dbname = String.IsNullOrEmpty(dbname)? "condStudyBDF3" : dbname; 
string table_name = String.Concat(buildname, "_", dbname);
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = ExecutionQueues[4];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

In [5]:
int SpaceDim = 2;
double _partRad = 0.6;
var Gshape = Shape.Sphere;

In [6]:
// set parameterz
int[] core_sweep = { 1};
int[] PolyDegS = new int[] {1, 2, 3};
int[] ResArray = new int[] {30};
double[] aggThresolds = new double[] {0, 0.1, 0.2, 0.3, 0.4, 0.5};

In [7]:
int NoOfTimeSteps = 80;
bool Steady = false;
bool IncludeConvection = true;

# Grid

In [8]:
double xMax = 1, yMax = 1, zMax = 1;
double xMin = -1, yMin = -1, zMin = -1;


In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();

foreach (var Res in ResArray) {
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);    
    GridCommons grd;
    string gname = SpaceDim == 2 ? "VanishingSphere2D_1BCs" : "VanishingSphere3D_1BCs";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;

            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells == (_xNodes.Length - 1) * (_yNodes.Length - 1) * (SpaceDim == 3 ? GenericBlas.Linspace(zMin, zMax, Res + 1).Length - 1 : 1);
            if(IsMatch) 
            tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }

    Console.WriteLine("Found {0} grids for Res={1}", tmp.Count, Res);
    if (tmp.Any()) {
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.First());
    } else {
        if (SpaceDim == 3) {
            grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
        } else {
            grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);
        }

        grd.Name = gname;
        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
        myDB.SaveGrid(ref grd);
        Grids.Add(grd);
    }
}

Console.WriteLine("\nThe grids to be simulated:");
Grids


## Generate Control object

### governing equations
- incompressible transient Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t) = \vec{u}_{body} = \boldsymbol{\omega}(t) \times \vec{r} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$
<br>$p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 



- angular velocity of rotating sphere $\boldsymbol{\omega}(t)=\frac{Re*\mu_A}{\rho_A*r_{p}*(2*r_p)}$

In [10]:
Func<IGridInfo, int,double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = 
delegate(IGridInfo grd, int k, double viscosity, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;

    //Session Settings
    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    C.ImmediatePlotPeriod = 1;
    C.SuperSampling = 0;    

    // Physical Parameters
    // =================== 
    const double rhoA = 1;
    const double Re = 500;
    double muA = viscosity;
    double partRad = _partRad;
    double partDia = 2*partRad;
    double VelocityMax = Re * muA / rhoA / partDia;
    double anglev = VelocityMax / partRad;
    double period = 2*Math.PI/anglev;
    double ts = period/NoOfTimeSteps;
    double[] pos = new double[SpaceDim];
    string rotAxis= RotAxis ?? "z";
    
    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    // Meta data
    // ============    
    C.SessionName = string.Format("J{0}_k{1}_agg{2}_AMR{3}_LB{4}_at{5}_r{6}_W{7:f2}_{8}D",
                     J, k,AggThrs,AMR,LoadBalance,rotAxis.ToUpper(),partRad,anglev,SpaceDim);

    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees and configuration
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 8)); // (velocity degree,level set) (pressure degree = k-1)
    C.saveperiod = 10;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = false;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;

    // rigid body and boundary conditions
    // ===================
    double rateOfRadius = Steady ? 0 : -1/(double)NoOfTimeSteps/ts;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim, rateOfRadius: rateOfRadius, staticShape: Steady);
    C.SessionName += $"dRdt{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis(rotAxis);
    C.UseImmersedBoundary = true;
    //C.Rigidbody.AssignExactSolutionForSteadyRotatingSphere();

    // Set initial and boundary conditions
    // ============
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");

    // solver options
    // ===================
    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None; // idle level set (used for evolving LS)
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 0;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;
    C.AdaptiveMeshRefinement = AMR;

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = Double.MaxValue;
        C.NoOfTimesteps = 1;
        C.SessionName += "_Steady";
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps; 
    }
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3; 
    C.dtFixed = dt;
    C.SkipSolveAndEvaluateResidual = false; //to save computational time for condition study
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;
    C.PlotAgglomeration = false;

    // Condition number study
    // ============
    var config = new OperatorAnalysisConfig();
    config.CalculateMassMatrix = true;
    config.CalculateGlobalConditionNumbers = true;
    config.CalculateStencilConditionNumbers = false;
    config.PlotStencilConditionNumbers = false;
    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
    C.SessionName += "_condStudy";

    return C;    
};

In [11]:
//GenXNSECtrl(Grids[0],1,1,"z",false,false,0)

# Submit jobs into cluster
Create the control objects for the sessions

In [12]:
var ViscositySweep = new double[]{1E-2};
var axes = new string[]{"z"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};

List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids)
    foreach(int k in PolyDegS)
            foreach(double v in ViscositySweep)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
            {
                controls.Add(GenXNSECtrl(grd,k,v,axis,AMR,LoadBalance,agg));
            }


List sessions to be submitted

In [ ]:
Console.WriteLine($"Total Number of Control Files: {controls.Count}");
controls.Summary()

Submit jobs

In [ ]:
int k = 0;
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores;            
            var aJob   = new Job("Vanishing"+Gshape+"_"+ctrl.SessionName,typeof(XNSE));
            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            //aJob.ExecutionTime            = "72:00:00";
            aJob.UseComputeNodesExclusive = true;
            aJob.NumberOfThreads = 4;
            aJob.Activate(myBatch);
            
            Console.WriteLine("Submitted {0}th \n",k) ;
            k++;           
        }       
    }


List sessions submitted

In [ ]:
myDB.Sessions